# Задание 1: EM-алгоритм

* Открыть датасет sklearn.datasets.load_wine, содержащий информацию о трех различных сортах вина (class0, class1, class2). Ответить на вопросы ниже, используя средства языка Python и необходимых библиотек;
* Используя файл Sem5_EM.ipynb, модифицировать алгоритм EM так, чтобы он умел распознавать три класса (в исходной реализации умеем делать только бинарную классификацию);
* Вместо переменной steps (указывает количество итераций алгоритма) и цикла по количеству шагов, сделать функцию FullEM(), которая будет выполнять функции e_step() и m_step(), пока не будет соблюдено условие сходимости (которое Вы выбираете сами). Таким образом, алгоритму не надо делать ровно 15 итераций, а их количество динамически зависит от условия сходимомсти;
* Не используя деление выборки на train-test (так как обучение без учителя), прогнать модифицированный EM-алгоритм (функцию FullEM()) и посчитать известные метрики точности классификации (спойлер: не только accuracy).
* Использовать GaussianMixture из sklearn, также посчитать метрики. Насколько точна классификация в данном случае? Какой из методов оказался точнее?

In [155]:
from sklearn.datasets import load_wine
from sklearn.mixture import GaussianMixture as GMM
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy.stats import multivariate_normal
from sklearn.metrics import accuracy_score, classification_report

Загружаю датасет

In [156]:
wd = load_wine()
wine = pd.DataFrame(wd.data, columns=wd.feature_names)
wine['target'] = wd.target
wine

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [159]:
data = wd.data
scaled = StandardScaler().fit_transform(data)
k = 3
n, m = scaled.shape

E-шаг

In [160]:
def e_step(input_data, weights, means, covariances):
    num_data_points, num_clusters = input_data.shape[0], len(weights)
    responsibilities = np.zeros((num_data_points, num_clusters))
    for i, (weight, mean, cov) in enumerate(zip(weights, means, covariances)):
        responsibilities[:, i] = weight * multivariate_normal.pdf(input_data, mean=mean, cov=cov)
    responsibilities /= np.sum(responsibilities, axis=1)[:, np.newaxis]
    return responsibilities

M-шаг

In [161]:
def m_step(data, assigned_cluster):
    num_data_points, num_features = data.shape
    num_clusters = assigned_cluster.shape[1]
    cluster_weights = np.zeros(num_clusters)
    cluster_means = np.zeros((num_clusters, num_features))
    cluster_covariances = np.zeros((num_clusters, num_features, num_features))

    for cluster in range(num_clusters):
        weight_sum = np.sum(assigned_cluster[:, cluster])
        cluster_weights[cluster] = weight_sum / num_data_points
        weighted_sum = np.dot(np.reshape(assigned_cluster[:, cluster], (-1, 1)).T, data).reshape(-1)
        cluster_means[cluster] = weighted_sum / weight_sum
        diff = data - cluster_means[cluster]
        cluster_covariances[cluster] = np.dot((np.reshape(assigned_cluster[:, cluster], (-1, 1)) * diff).T, diff) / weight_sum

    return cluster_weights, cluster_means, cluster_covariances

full_em

In [168]:
def full_em(inputdata, maxiterations=100, tolerance=1e-4):
    wts = np.full(k, 1 / k)
    # print(k, m)
    avg = np.random.random((k, m))
    cov = np.array([np.eye(m)] * k)
    log_val = 0
    for iteration in range(maxiterations):
        before = log_val
        resp = e_step(inputdata, wts, avg, cov)
        wts, avg, cov = m_step(inputdata, resp)
        log_val = np.sum(np.log(np.sum([w * multivariate_normal.pdf(inputdata, mean=avg[i], cov=cov[i]) for i, w in enumerate(wts)], axis=0)))

        if abs(log_val - before) < tolerance:
            break

    return wts, avg, cov

Точность классификации

In [300]:
weights, means, covs = full_em(scaled)
output = False
if output:
    print("Averages:")
    for i in means:
        print(f"\t{i}")

    print("\nWeights:")
    for i in weights:
        print(f"\t{i}")

    print("\nCovariance:")
    for i in covs:
        print(f"\t{i}")

results = []
for i in range(k):
    results.append(multivariate_normal.pdf(scaled, mean=means[i], cov=covs[i]))
predict = np.argmax(results, axis=0)

# print(predicted_classes)
print("Отчёт классификации EM-флгоритма:")
print(classification_report(wine.target, predict))

print("Точность EM-флгоритма:")
print(f"\t{accuracy_score(wine.target, predict)}")

Отчёт классификации EM-флгоритма:
              precision    recall  f1-score   support

           0       0.44      0.25      0.32        59
           1       0.54      0.73      0.62        71
           2       1.00      1.00      1.00        48

    accuracy                           0.65       178
   macro avg       0.66      0.66      0.65       178
weighted avg       0.63      0.65      0.62       178

Точность EM-флгоритма:
	0.6460674157303371


In [327]:
test = GMM(n_components=k, covariance_type='full', random_state=0).fit(scaled)
predict2 = test.predict(scaled)

print("Отчёт классификации:")
print(classification_report(wine.target, predict2))

print("Точность:")
print(f"\t{accuracy_score(wine.target, predict2)}")


Отчёт классификации:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.06      0.04      0.05        71
           2       0.00      0.00      0.00        48

    accuracy                           0.02       178
   macro avg       0.02      0.01      0.02       178
weighted avg       0.02      0.02      0.02       178

Точность:
	0.016853932584269662


### Выводы

Модифицированный алгоритм показал точность 0.6460674157303371, тогда как готовая реализация sklearn всего 0.016853932584269662
Можно сказать, что готовая реализация модели не справляется с классификацией данных

Точность по классам:

| Класс | Самопильный метод | Готовый |
|-------|-------------------|---------|
| 1     | 0.44              | 0.00    |
| 2     | 0.54              | 0.06    |
| 3     | 1.00              | 0.00    |


## Задание 2: kNN

1) Используем датасет с сортами вина из предыдущей задачи;
2) Использовать три подхода к делению выборки на тренировочную и тестовую: KFold, LOO, Stratified KFold. Для воспроизводимости зафиксируйте параметр random_state=42;
3) Для каждого из методов кросс-валидации, а также для каждого k ∈ [1, 50] (число "соседей") прогнать алгоритм ближайших соседей (sklearn.neighbors.KNeighborsClassifier) и посчитать долю правильных ответов. Какая кросс-валидация и при каком значении k дает лучший результат?;
4) Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. Снова найдите оптимальное k на трех разных кросс-валидациях. Чему оно равно? Изменилось ли оно? Изменился ли оптимальный методвалидации?

**Шаг 1: Загрузка и Подготовка Датасета**

Сначала загрузим датасет load_wine из sklearn.datasets и подготовим данные для дальнейшего анализа.

In [11]:
# Импортирую необходимые библиотеки
from sklearn.datasets import load_wine
import pandas as pd

# Загружаю датасет
wd = load_wine()
data = wd.data
y = wd.target

# Преобразую данные в DataFrame для удобства
wine = pd.DataFrame(data, columns=wd.feature_names)
wine['target'] = y

# Вывожу первые несколько строк датасета для ознакомления
print(wine.head())


   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  target  
0          

**Шаг 2: Разделение Данных с Использованием Различных Методов Кросс-Валидации**

Применим три различных метода кросс-валидации: KFold, LOO (Leave One Out), и Stratified KFold.

In [12]:
# Импортирую методы кросс-валидации
from sklearn.model_selection import KFold, LeaveOneOut, StratifiedKFold

# Инициализирую методы кросс-валидации
kf = KFold(n_splits=5, random_state=42, shuffle=True)
loo = LeaveOneOut()
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)


**Шаг 3: Применение kNN и Подсчет Доли Правильных Ответов**

Применим алгоритм k-ближайших соседей для каждого k в диапазоне от 1 до 50 и каждого метода кросс-валидации.

In [13]:
# Импортирую KNeighborsClassifier и cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Определяю функцию для тестирования kNN с различными параметрами k и методами кросс-валидации
def test_knn_cv(X, y, cv_method, k_range):
    scores = {}
    for k in k_range:
        model = KNeighborsClassifier(n_neighbors=k)
        cv_scores = cross_val_score(model, X, y, cv=cv_method)
        scores[k] = np.mean(cv_scores)
    return scores

# Тестирую для каждого метода кросс-валидации
k_range = range(1, 51)
scores_kf = test_knn_cv(data, y, kf, k_range)
scores_loo = test_knn_cv(data, y, loo, k_range)
scores_skf = test_knn_cv(data, y, skf, k_range)


**Шаг 4: Анализ Результатов и Определение Лучшей Конфигурации**

Я проанализировал полученные результаты, чтобы определить, какая комбинация кросс-валидации и значения k даёт наилучший результат.

In [14]:
# Нахожу наилучшие результаты для каждого метода кросс-валидации
best_k_kf = max(scores_kf, key=scores_kf.get)
best_k_loo = max(scores_loo, key=scores_loo.get)
best_k_skf = max(scores_skf, key=scores_skf.get)

print(f"Лучшее k для KFold: {best_k_kf} с точностью {scores_kf[best_k_kf]}")
print(f"Лучшее k для LOO: {best_k_loo} с точностью {scores_loo[best_k_loo]}")
print(f"Лучшее k для Stratified KFold: {best_k_skf} с точностью {scores_skf[best_k_skf]}")


Лучшее k для KFold: 1 с точностью 0.7304761904761905
Лучшее k для LOO: 1 с точностью 0.7696629213483146
Лучшее k для Stratified KFold: 1 с точностью 0.7185714285714285


Эти результаты показывают, что для необработанных данных наибольшую точность дает наименьшее значение k (1), что характерно для kNN, особенно на небольших или менее сложных наборах данных.

**Шаг 5: Масштабирование Признаков и Повторное Тестирование**

После масштабирования признаков снова применим алгоритм kNN для каждого k в диапазоне от 1 до 50 и каждого метода кросс-валидации. Целью является сравнить результаты до и после масштабирования и определить оптимальные значения k.

In [18]:
# Масштабирование признаков
from sklearn.preprocessing import scale
scaled = scale(data)

# Повторяем тестирование с масштабированными данными
scores_kf_scaled = test_knn_cv(scaled, y, kf, k_range)
scores_loo_scaled = test_knn_cv(scaled, y, loo, k_range)
scores_skf_scaled = test_knn_cv(scaled, y, skf, k_range)

# Находим наилучшие результаты для каждого метода кросс-валидации после масштабирования
best_k_kf_scaled = max(scores_kf_scaled, key=scores_kf_scaled.get)
best_k_loo_scaled = max(scores_loo_scaled, key=scores_loo_scaled.get)
best_k_skf_scaled = max(scores_skf_scaled, key=scores_skf_scaled.get)

print(f"Лучшее k для KFold (масштабированные данные): {best_k_kf_scaled} с точностью {scores_kf_scaled[best_k_kf_scaled]}")
print(f"Лучшее k для LOO (масштабированные данные): {best_k_loo_scaled} с точностью {scores_loo_scaled[best_k_loo_scaled]}")
print(f"Лучшее k для Stratified KFold (масштабированные данные): {best_k_skf_scaled} с точностью {scores_skf_scaled[best_k_skf_scaled]}")


Лучшее k для KFold (масштабированные данные): 29 с точностью 0.9776190476190475
Лучшее k для LOO (масштабированные данные): 36 с точностью 0.9831460674157303
Лучшее k для Stratified KFold (масштабированные данные): 13 с точностью 0.9776190476190475


После масштабирования признаков точность увеличилась, а оптимальные значения k сместились в сторону более высоких значений. Это подтверждает, что масштабирование признаков может улучшить производительность алгоритмов машинного обучения для методов, основанных на расстояниях, как kNN.

**Шаг 6: Анализ Изменений и Выводы**

Сравним результаты до и после масштабирования, чтобы понять, изменилось ли оптимальное значение k и какой метод валидации оказался наиболее эффективным.

In [16]:
# Сравнение результатов до и после масштабирования
print(f"Изменение оптимального k для KFold: с {best_k_kf} на {best_k_kf_scaled}")
print(f"Изменение оптимального k для LOO: с {best_k_loo} на {best_k_loo_scaled}")
print(f"Изменение оптимального k для Stratified KFold: с {best_k_skf} на {best_k_skf_scaled}")


Изменение оптимального k для KFold: с 1 на 29
Изменение оптимального k для LOO: с 1 на 36
Изменение оптимального k для Stratified KFold: с 1 на 13


Ответы на Вопросы

1) Какая кросс-валидация и при каком значении k дает лучший результат?

Перед масштабированием данных лучший результат показал метод LOO с k = 1. После масштабирования лучший результат показал метод LOO с k = 36.

2) Чему равно оптимальное k на трех разных кросс-валидациях после масштабирования? Изменилось ли оно? Изменился ли оптимальный метод валидации?
        
Оптимальное k после масштабирования: для KFold = 29, для LOO = 36, и для Stratified KFold = 13.
Да, оптимальное k изменилось после масштабирования, увеличившись в каждом случае.
Метод валидации, который показал наилучший результат, остался прежним (LOO), но оптимальное значение k изменилось.

Таким образом, масштабирование признаков повлияло на производительность алгоритма kNN, позволяя достичь более высокой точности с более высокими значениями k.